# 💳 Finanzas y Banca: riesgo, fraude, rentabilidad
KPIs: NPL, PD/LGD/EAD, NIM, churn, CAC, fraude por canal.

## Casos de uso
- Scoring de crédito y límites dinámicos
- AML/KYC, detección de anomalías transaccionales
- Segmentación RFM y propensión a compra
- Monitoreo de liquidez y escenarios regulatorios (IFRS9/Basilea)

In [ ]:
# Mini ejemplo PD con features simples (toy)
def prob_default(score: float) -> float:
    import math
    z = -2.0 + 0.03 * score
    return 1/(1+math.exp(-z))

[round(prob_default(s),3) for s in [300,500,700]]

## Arquitectura
- Flujos transaccionales en streaming
- Feature store para variables de riesgo
- Modelos gobernados, explainability y trazabilidad
- Data contracts y auditoría regulatoria

## 📌 Valor y cumplimiento
- Orígenes consolidados y trazables → auditorías y reporting regulatorio.
- Modelos de riesgo consistentes (features versionadas) → decisiones rápidas.
- Prevención de fraude y AML → pérdidas evitadas y reputación.
- Costos optimizados en nube con políticas de datos fríos/calientes.


In [ ]:
# Práctica: PD y Pérdida Esperada (ECL) en toy data
import pandas as pd

clientes = pd.DataFrame({
    'score': [300, 450, 600, 700, 800],
    'EAD': [1000, 2000, 5000, 8000, 12000],  # exposición
    'LGD': [0.6, 0.5, 0.45, 0.4, 0.35],      # pérdida dado default
})

import math
pd_func = lambda s: 1/(1+math.exp(-(-2.0 + 0.003*s)))
clientes['PD'] = clientes['score'].apply(pd_func)
clientes['ECL'] = clientes['PD'] * clientes['LGD'] * clientes['EAD']
print(clientes[['score','PD','EAD','LGD','ECL']])
print('ECL total:', round(clientes['ECL'].sum(), 2))


## 🏗️ Arquitectura y gobierno
- Ingesta transaccional + streaming; particionado por fecha y cliente.
- Feature store para riesgo (ventanas, agregaciones, drifts monitorizados).
- Lineage, versionado de modelos y explainability; alertas de sesgo.
- Controles de acceso por dominio y tokenización de PII.


In [ ]:
# Visual: distribución ECL (toy)
import plotly.express as px
fig = px.bar(clientes, x='score', y='ECL', title='Pérdida Esperada (ECL) por score')
fig.show()


## ✅ Checklist de estrategia de negocio y datos
- Objetivo financiero (riesgo, NIM, crecimiento cartera): …
- KPIs: PD/LGD/EAD, NPL, fraude, churn.
- Palancas: feature store riesgo, modelos gobernados, AML/KYC.
- Datos: transaccionales, bureaus, señales alternas; calidad/lineage.
- Gobierno: acceso por dominio, tokenización PII, auditoría.
- SLOs: latencia scoring, costo/consulta, disponibilidad.
- ROI/supuestos: …
- Riesgos: sesgos, drift, cumplimiento regulatorio.


## 🔗 Puente Estrategia ↔ Ingeniería de Datos
- OKR: Fraude ≤ 0.1% y NPL estable → KPI negocio: pérdidas por fraude, NPL → KPI datos: latencia scoring <200ms, cobertura features ≥99% → Capacidades: streaming + feature store + explainability → Decisión: bloqueo/aprobación en tiempo real → Impacto: pérdidas evitadas.

```
[Transacciones]→[Streaming]→[Features riesgo]→[Score]→[Umbral]→[Bloqueo/Aprobación]→[Δ pérdidas]
```


In [ ]:
# contrato_de_datos (finanzas)
contrato_datos = {
    "objetivo_negocio": "Fraude ≤ 0.1%",
    "kpi_negocio": "Pérdidas por fraude",
    "kpi_datos": {"latencia_ms": "<200", "cobertura_features": ">=99%"},
    "dataset": ["transacciones", "clientes", "dispositivos"],
    "propietario": "Dominio Riesgo",
    "slo": {"drift_alertas_dia": "<=1"},
    "decision": "Bloqueo/aprobación real time",
    "impacto_economico_usd": 300000,
}
print(contrato_datos)


## 📋 Caso de uso: Detección de fraude en tiempo real

**Contexto de negocio:** Un banco pierde $4M/año en fraude transaccional. El área de Riesgo necesita bloquear transacciones sospechosas en <200ms sin afectar legítimas (tasa de rechazo falso <0.5%).

**Problema sin ingeniería de datos:**
- Scoring batch: detecta fraudes 24h después (dinero ya transferido).
- Features calculadas manualmente en Excel, sin versionado.
- Modelo entrenado con datos desactualizados (6 meses atrás).

**Solución con ingeniería de datos:**
1. Pipeline streaming: Kafka ingesta transacciones en tiempo real.
2. Feature store: 50+ features pre-calculadas (ventanas móviles, ubicación, device fingerprint) con <10ms de latencia.
3. Modelo ML servido vía API con SLA <200ms.
4. Feedback loop: transacciones confirmadas como fraude re-entrenan el modelo semanalmente.
5. Lineage completo: se puede auditar qué features produjeron cada decisión.

**Resultado:**
- Detección: 24h → <200ms.
- Tasa de detección: 65% → 89%.
- Falsos positivos: 2.1% → 0.4%.
- Ahorro anual estimado: $3.2M + mejora en NPS por menos rechazos legítimos.


In [ ]:
# Ejercicio: feature engineering para riesgo de crédito
import pandas as pd
import numpy as np

# Simular transacciones con features
np.random.seed(42)
transacciones = pd.DataFrame({
    'cliente_id': range(1, 11),
    'monto': [100, 5000, 200, 15000, 50, 8000, 300, 12000, 150, 20000],
    'hora': [14, 3, 10, 23, 12, 2, 15, 22, 11, 1],
    'device_nuevo': [False, True, False, True, False, True, False, True, False, True],
})

# Feature engineering (clave en ingeniería de datos)
transacciones['es_horario_nocturno'] = (transacciones['hora'] < 6) | (transacciones['hora'] > 22)
transacciones['monto_alto'] = transacciones['monto'] > 10000
transacciones['score_riesgo'] = (
    transacciones['es_horario_nocturno'].astype(int) * 30 +
    transacciones['monto_alto'].astype(int) * 50 +
    transacciones['device_nuevo'].astype(int) * 20
)

print("Features de riesgo calculadas:")
print(transacciones[['cliente_id', 'monto', 'hora', 'score_riesgo']].head())
print(f"\nTransacciones de alto riesgo (score>50): {(transacciones['score_riesgo'] > 50).sum()}")
print("💡 Feature store permite pre-calcular estas features con <10ms de latencia")
